# <center> Demo Inference </center>

In [12]:
import random
from functools import partial
from omegaconf import DictConfig
import os
import torch
import rasterio
import numpy as np
from pathlib import Path
import h5py
from typing import Dict, Any
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)   # ou FutureWarning, UserWarning, ...

import argparse
import json
import os
import pprint
import sys
from pathlib import Path
import torch
from omegaconf import OmegaConf
from torch.utils.tensorboard import SummaryWriter

from data.uncrtaints_adapter import UnCRtainTS_CIRCA_Adapter
from dataloader_CIRCA_old.datasets.CIRCA_dataset_for_UnCRtainTS import CircaPatchDataSetForUnCRtainTS
from torch.utils.tensorboard import SummaryWriter
os.chdir(os.path.join(os.getcwd(), "model"))

from parse_args import create_parser
from src.model_utils import get_model, load_checkpoint
from src import utils
from train_reconstruct import (
    iterate,
    prepare_output,
    save_results,
    seed_packages,
)

if os.getcwd().endswith("model"):
    os.chdir(os.path.dirname(os.getcwd()))

In [13]:
def prepare_output(config):
    os.makedirs(os.path.join(config.res_dir, config.experiment_name), exist_ok=True)

In [53]:
# Définition des chemins pour faire l'inférence
experiment_name = "UnCRtainTS_bs_7_MGNLL_experiment" 
res_dir = "/DATA_10TB/data_rpg/outputs/UnCRtainTS/inference"
weight_folder = "/DATA_10TB/data_rpg/outputs/UnCRtainTS/results/train"

parser = create_parser(mode="test")
test_config, _ = parser.parse_known_args()
test_config.pid = os.getpid()
test_config = OmegaConf.create(vars(test_config))
assert test_config.get("sample_type", False)

# Récupération des paramètres issus du modèle 
NO_OVERWRITE = ['pid', 'device', 'resume_at', 'trained_checkp', 'res_dir', 'weight_folder', 'root1', 'root2', 'root3', 'max_samples_count', 'batch_size', 'display_step', 'plot_every', 'export_every', 'input_t', 'region', 'min_cov', 'max_cov']
conf_path = os.path.join(weight_folder, experiment_name, "conf.json")
model_config = OmegaConf.create(json.load(open(conf_path)))
for key_to_delete in NO_OVERWRITE:
    if model_config.get(key_to_delete, False):
        model_config.pop(key_to_delete)
# Mise à jour des paramètres de base du parser par les paramètres pris pour la création du modèle utilisé pour l'inférence 
test_config.update(model_config)
test_config.update(
    {
        "experiment_name":experiment_name,
        "res_dir": res_dir,
        "weight_folder": weight_folder,
    }
)
test_config = utils.str2list(test_config, ["encoder_widths", "decoder_widths", "out_conv"])
config = OmegaConf.create(test_config)

In [54]:
S1_LAUNCH: str = "2014-04-03"
SEN12MSCRTS_SEQ_LENGTH: int = 30  # Length of the Sentinel time series
CLEAR_THRESHOLD: float = 1e-3  # Threshold for considering a scene as cloud-free
INPUT_T = 3 

device = torch.device(config.device)
config.use_sar = True
prepare_output(config)

model = get_model(config)
model = model.to(device)
config.N_params = utils.get_ntrainparams(model)
print(f"TOTAL TRAINABLE PARAMETERS: {config.N_params}\n")

# instantiate tensorboard logger
writer = SummaryWriter(os.path.join(config.res_dir, config.experiment_name))

circa_ds = UnCRtainTS_CIRCA_Adapter(
    phase="test",
    hdf5_file= "/DATA_10TB/data_rpg/circa/hdf5/CIRCA_CR_merged.hdf5",
    shuffle=False,
    use_sar="mix_closest",
    channels= "all",
    compute_cloud_mask=False,
    # paramaters specific to UnCRtainTS
    cloud_masks= "s2cloudless_mask",
    sample_type= "cloudy_cloudfree",
    sampler= "fixed",
    n_input_samples=INPUT_T,
    rescale_method= "default",
    min_cov= 0.0,
    max_cov= 1.0,
    ref_date= S1_LAUNCH,
    seq_length=30, #TODO comment rendre cela adaptatif à la donnée ? 
    clear_threshold=CLEAR_THRESHOLD,
    vary_samples=False, 
)

dt_test = torch.utils.data.Subset(
    circa_ds, range(0, min(config.max_samples_count, len(circa_ds)))
)

test_loader = torch.utils.data.DataLoader(
    dt_test, 
    batch_size=config.batch_size,
    shuffle=False,
    num_workers=1
    #collate_fn=circa_collate_fn  # Utiliser notre fonction personnalisée
)

TOTAL TRAINABLE PARAMETERS: 635412



In [55]:
chckp_path = os.path.join(config.weight_folder, config.experiment_name, f"model.pth.tar")
print(f"Loading checkpoint {chckp_path}")
checkpoint = torch.load(chckp_path, map_location=config.device)["state_dict"]

try:  # try loading checkpoint strictly, all weights & their names must match
    model.load_state_dict(checkpoint, strict=True)
    print("Loaded checkpoint with matching keys")
except:
    # rename keys
    #   in_block1 -> in_block0, out_block1 -> out_block0
    checkpoint_renamed = dict()
    for key, val in checkpoint.items():
        if "in_block" in key or "out_block" in key:
            strs = key.split(".")
            strs[1] = strs[1][:-1] + str(int(strs[1][-1]) - 1)
            strs[1] = ".".join([strs[1][:-1], strs[1][-1]])
            key = ".".join(strs)
        checkpoint_renamed[key] = val
    model.load_state_dict(checkpoint_renamed, strict=False)

# Inference
print("Testing . . .")
model.eval()

_, test_img_metrics = iterate(
    model,
    data_loader=test_loader,
    config=config,
    writer=writer,
    mode="test",
    epoch=1,
    device=device,
)
print(f"\nTest image metrics: {test_img_metrics}")

Loading checkpoint /DATA_10TB/data_rpg/outputs/UnCRtainTS/results/train/UnCRtainTS_bs_7_MGNLL_experiment/model.pth.tar
Loaded checkpoint with matching keys
Testing . . .


  0%|▌                                                                                                                                                                           | 2/594 [00:16<1:22:04,  8.32s/it]


KeyboardInterrupt: 

## Code obsolète

In [ ]:
# Définition des chemins pour faire l'inférence
# experiment_name = "UnCRtainTS_bs_7_MGNLL_experiment" 
# res_dir = "/DATA_10TB/data_rpg/outputs/UnCRtainTS/inference"
# weight_folder = "/DATA_10TB/data_rpg/outputs/UnCRtainTS/results/train"

# parser = create_parser(mode="test")
# test_config, _ = parser.parse_known_args()
# test_config.pid = os.getpid()

# print(test_config.sample_type)

# conf_path = os.path.join(weight_folder, experiment_name, "conf.json")
# print(conf_path)
# if os.path.isfile(conf_path):
#     with open(conf_path) as file:
#         model_config = json.loads(file.read())
#         t_args = argparse.Namespace()
#         # do not overwrite the following flags by their respective values in the config file
#         no_overwrite = [
#             "pid",
#             "device",
#             "resume_at",
#             "trained_checkp",
#             "res_dir",
#             "weight_folder",
#             "root1",
#             "root2",
#             "root3",
#             "max_samples_count",
#             "batch_size",
#             "display_step",
#             "plot_every",
#             "export_every",
#             "input_t",
#             "region",
#             "min_cov",
#             "max_cov",
#         ]
#         conf_dict = {
#             key: val for key, val in model_config.items() if key not in no_overwrite
#         }
        
#         for key, val in vars(test_config).items():
#             if key in no_overwrite:
#                 conf_dict[key] = val
#         t_args.__dict__.update(conf_dict)
#         config = OmegaConf.create(vars(t_args))
#         config.update({
#             "experiment_name":experiment_name,
#             "res_dir": res_dir,
#             "weight_folder": weight_folder,
#         })
        
# else:
#     config = test_config  # otherwise, keep passed flags without any overwriting
# config = utils.str2list(config, ["encoder_widths", "decoder_widths", "out_conv"])